In [77]:
import numpy as np
import glob, os, sys
import copy
import pandas as pd
import datetime
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select, Div
from bokeh.models import LabelSet
from bokeh.models.glyphs import MultiLine, Text
from bokeh.plotting import figure, output_file, show, output_notebook
from PIL import Image

In [133]:
def make_combined_df(date, members):
    date_label = date.strftime("%Y%m%d")
    bsiso_archive_dir = os.path.join('/project/MJO_GCSS/SEA_monitoring/processed_BSISO_data/glosea/bsiso_new/',
                                     f'{date_label}')
    dfs = []
    for m, mem in enumerate(members):
        bsiso_index_file_name = os.path.join(bsiso_archive_dir,
                                             f'BSISO.{date_label}.fcast.{mem}.txt')
        # Concatenate all DataFrames in the list along rows
        pcs = pd.read_csv(bsiso_index_file_name)
        # Append the computed PCs for the current member to the list
        dfs.append(pcs)

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df
    
def make_ens_mean_df(combined_df):
    ens_mean_df = combined_df.groupby('date', as_index=False)[['PC1', '8_phases', 'PC2', 'amp']].mean()
    return ens_mean_df

In [141]:
date = datetime.datetime(2024, 5, 14)
nanalysis2write = 40
nforecasts = 30
members = [str('%03d' % mem) for mem in range(4)]
combined_df = make_combined_df(date, members)
ens_mean_df = make_ens_mean_df(combined_df)
#ens_mean_df.loc[ens_mean_df.date.todatetime() < date]
len(ens_mean_df)

70

In [142]:
im = Image.open('../data/BSISO_Phases_BG.png')

imarray = np.array(im.convert("RGBA"))
imarray = imarray[::-1]

hover = HoverTool(tooltips=[
    ("Date", "@date"),
    ("PC1", "@PC1"),
    ("PC2", "@PC2"),
    ("Phase", "@8_phases"),
    ("Amp", "@amp"),
    ("Label", "@label"),
    ("Member", "@member")
], mode='mouse')

p = figure(width=500, height=500, x_range=[-4, 4], y_range=[-4, 4], tools=["pan, reset, save, wheel_zoom, box_zoom", hover])
p.image_rgba(image=[imarray.view("uint32").reshape(imarray.shape[:2])], x=-4.275, y=-4.275, dw=8.525, dh=8.52, alpha=0.8)

for mem in members:
    df = combined_df.loc[combined_df.mem==int(mem)][-(nforecasts):]
    p.line('PC1', 'PC2', alpha=0.3, source=df)

p.line('PC1', 'PC2', alpha=0.8, color='grey',line_width=3, source=ens_mean_df[-(nforecasts+5):-(nforecasts)])
p.circle('PC1', 'PC2', alpha=0.8, color='grey',line_width=3, source=ens_mean_df[-(nforecasts+5):-(nforecasts)])
p.line('PC1', 'PC2', alpha=0.8, line_width=3, source=ens_mean_df[-(nforecasts):])
p.circle('PC1', 'PC2', radius=0.05, alpha=0.8, source=ens_mean_df[-(nforecasts):])
show(p)

In [71]:
4.275*2

8.55

In [19]:
imarray = np.array(im.convert("RGBA"))
imarray = imarray[::-1, ::-1]

In [7]:
rmms_archive_dir = '/project/MJO_GCSS/MJO_monitoring/processed_MJO_data/mogreps/rmms_new/20240109'

In [8]:
date = datetime.datetime(2024, 1, 9)
# All ensemble members
members = [str('%03d' % mem) for mem in range(36)]

In [9]:
rmm_file_names = [os.path.join(rmms_archive_dir, f'createdPCs.15sn.{date.strftime("%Y%m%d")}.fcast.{mem}.txt') for mem in members]

In [10]:
df = pd.read_csv(rmm_file_names[0])
df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')

FileNotFoundError: [Errno 2] No such file or directory: '/project/MJO_GCSS/MJO_monitoring/processed_MJO_data/mogreps/rmms_new/20240109/createdPCs.15sn.20240109.fcast.000.txt'

In [ ]:
def make_ens_mean_df(rmm_file_names):
    # read one file
    df = pd.read_csv(rmm_file_names[0])
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
    # Make dummy for ensemble mean
    ens_mean_df = df.copy()
    # Loop through each CSV file, read it into a DataFrame, and concatenate to the combined_df
    for col in ['rmm1', 'rmm2', 'amp']:
        ens_mean_df[col] = np.array([pd.read_csv(file)[col] for file in rmm_file_names]).mean(axis=0)
    # Phase has to be integer
    ens_mean_df['phase'] = [int(x) for x in np.array([pd.read_csv(file)['phase'] for file in rmm_file_names]).mean(axis=0)]
    return ens_mean_df

In [ ]:

#ens_mean_df

In [ ]:
# Set up plot
hover = HoverTool(tooltips=[
    ("Date", "@date"),
    ("RMM1", "@rmm1"),
    ("RMM2", "@rmm2"),
    ("Phase", "@phase"),
    ("Amp", "@amp"),
    ("Label", "@label"),
    ("Member", "@member")
], mode='mouse')

ens_mean_df = make_ens_mean_df(rmm_file_names)

def make_plot(title=None):
    '''
    # Generates the axes and background for the data to be plot on
    #
    :param title:
    :return:
    '''
    plot = figure(height=500, width=500, tools=["pan, reset, save, wheel_zoom, box_zoom", hover],
                     x_range=[-4, 4], y_range=[-4, 4])

    plot.title.text = title

    # Mark the 8 sectors
    x = 4
    y = 0.707107
    linewidth = 0.25
    plot.line([-x, -y], [-x, -y], line_width=0.5, line_alpha=0.6)
    plot.line([y, x], [y, x], line_width=0.5, line_alpha=0.6)
    plot.line([-x, -y], [x, y], line_width=0.5, line_alpha=0.6)
    plot.line([y, x], [-y, -x], line_width=0.5, line_alpha=0.6)
    plot.line([-x, -1], [0, 0], line_width=0.5, line_alpha=0.6)
    plot.line([1, x], [0, 0], line_width=0.5, line_alpha=0.6)
    plot.line([0, 0], [-x, -1], line_width=0.5, line_alpha=0.6)
    plot.line([0, 0], [1, x], line_width=0.5, line_alpha=0.6)

    xt, yt = 3., 1.5
    phase_marker_source = ColumnDataSource(data=dict(xt=[-xt, -yt, yt, xt, xt, yt, -yt, -xt],
                                                     yt=[-yt, -xt, -xt, -yt, yt, xt, xt, yt],
                                                     phase_labels=[str(i) for i in range(1, 9)]))
    labels = LabelSet(x='xt', y='yt', text='phase_labels', level='glyph',
                      x_offset=0, y_offset=0, source=phase_marker_source,
                      text_color='grey', text_font_size="30pt", text_alpha=0.25)

    plot.add_layout(labels)
    plot.circle([0], [0], radius=1, color="white", line_color='grey', alpha=0.6)

    phase_name_source = ColumnDataSource(dict(x=[0, 0], y=[-3.75, 3.], text=['Indian \n Ocean', 'Western \n Pacific']))
    glyph = Text(x="x", y="y", text="text", angle=0., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    phase_name_source = ColumnDataSource(dict(x=[-3.], y=[0], text=['West. Hem\n Africa']))
    glyph = Text(x="x", y="y", text="text", angle=np.pi / 2., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    phase_name_source = ColumnDataSource(dict(x=[3.], y=[0], text=['Maritime\n continent']))
    glyph = Text(x="x", y="y", text="text", angle=-np.pi / 2., text_color="grey", text_align='center', text_alpha=0.25)
    plot.add_glyph(phase_name_source, glyph)

    plot.xaxis[0].axis_label = 'RMM1'
    plot.yaxis[0].axis_label = 'RMM2'

    return plot

df_analysis = df.loc[df['label']=='analysis']
plot_mog = make_plot(title=f'MOGREPS MJO Forecasts {date.strftime("%Y-%m-%d")}')
plot_mog.line('rmm1', 'rmm2', source=df_analysis, name="analysis", line_color='grey', line_width=5, line_alpha=0.8)
plot_mog.circle('rmm1', 'rmm2', source=df_analysis, name="analysis_dots", color='grey', radius=0.05,
               alpha=0.8)

for mem, rmm_file_name in enumerate(rmm_file_names):
    df = pd.read_csv(rmm_file_name)
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
    fcast_start_index = min(df.loc[df['label']=='forecast'].index)
    # connect the forecasts to the analysis
    df_forecast = df.iloc[fcast_start_index-1:]
    
    # Add member info
    df_forecast['member'] = [mem for i in range(len(df_forecast))]
    
    plot_mog.line('rmm1', 'rmm2', source=df_forecast, name="analysis", line_color='lightskyblue', line_width=2, line_alpha=0.5)
    plot_mog.circle('rmm1', 'rmm2', source=df_forecast, name="analysis_dots", color='lightskyblue', radius=0.02,
                   alpha=0.5)

# Plot ensemble mean
fcast_start_index = min(ens_mean_df.loc[ens_mean_df['label']=='forecast'].index)
# connect the forecasts to the analysis
ens_mean_df = ens_mean_df.iloc[fcast_start_index-1:]
# Add member info
ens_mean_df['member'] = ['ens_mean' for i in range(len(ens_mean_df))]

plot_mog.line('rmm1', 'rmm2', source=ens_mean_df, name="analysis", line_color='navy', line_width=5, line_alpha=0.5)
plot_mog.circle('rmm1', 'rmm2', source=ens_mean_df, name="analysis_dots", color='navy', radius=0.05,
                   alpha=0.3)

# Specify the output file
output_file("/net/home/h03/hadpx/public_html/Monitoring_dashboard/mjo/plot.html")

# Display the plot
show(plot_mog)

In [ ]:

#rmm1s.shape

In [ ]:
ens_mean_df = make_ens_mean_df(df)
ens_mean_df

In [153]:
results = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [160]:
x = [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]

In [161]:
all(x)

True